In [1]:
from os import walk
import json
import re
from tqdm import tqdm
from utils import pmi, pmi_to_dict_adj, docs_distribution, new_distribution, OptimizedIncrementalPMI
from scoring import compute_scores
import time
import pandas as pd
from textCleaning import get_file_names, extract_year_ipc
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import utils
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import copy
from novelty import ClusterKS

import importlib
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import scoring
importlib.reload(scoring)
from scoring import compute_scores
import novelty
importlib.reload(novelty)

c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edgar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'novelty' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\novelty\\novelty.py'>

In [ ]:
columns = ['summary', 'abstract', 'background']
w_size = 3
path = "/home/edgarlanoue/data/csvCleaned/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
pathOutput = "/home/edgarlanoue/metrics/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"
useClusters=True
yearList = []
ipcList = []

def measureNov(path, pathOutput, columns, w_size, useClusters, yearList, ipcList):
    
    start_time = time.time()  # Record the starting time
    if yearList == [] or ipcList == []:
        tE_names = get_file_names(path+"tE")
        KS_names = get_file_names(path+"KS")
        ES_names = get_file_names(path+"ES")
    
        tE_set = set([extract_year_ipc(string) for string in tE_names])
        KS_set = set([extract_year_ipc(string) for string in KS_names])
        ES_set = set([extract_year_ipc(string) for string in ES_names])
    
        assert(tE_set==KS_set==ES_set)
    
        yearList = sorted(list(set([year for year, ipc in list(tE_set)])))
        ipcList = list(set([ipc for year, ipc in list(tE_set)]))
        print(yearList)
        print(ipcList)
    
    for year in yearList:
        for ipc in ipcList:
            print(f"{year}")
            print(f"    {ipc}")
            tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
            KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
            # ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")
    
            # patent = tE[k]
            # file_path = my_path + recette
            # with open(file_path) as json_file:
            #     recipe_dict = json.load(json_file)
    
            #### COLLECTING ALL NECESSARY INFO
    
            # KB_recettes, _ = data_analysis(recipe_dict)
            # print('Knowledge base size : ', len(KB_recettes))
            # if len(KB_recettes) <= 0:
            #     continue
    
            ### Transforming KB into distribution
            application_number = tE["application_number"]
            label = tE["label"]
    
            tE=combine_columns(tE, columns)
            KS=combine_columns(KS, columns)
            # ES=combine_columns(ES, columns)
    
            # Création pmi ES
            # ES_PMI = pmi(ES)
            
            # base_texts = [word for text in tqdm(ES) for word in text.split()]
            # instance_ES_pmi = OptimizedIncrementalPMI(window_size = w_size)
            # print("pmi")
            # instance_ES_pmi.update(base_texts)
            # ES_pmi = instance_ES_pmi.compute_pmi()
            
            # dict_ES_pmi = pmi_to_dict_adj(ES_pmi[0])
            
            KS_matrix, KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
            KS_size = list(range(KS_matrix.shape[0]))
            if useClusters==True:
                KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q= None, N=100, nbPtsPerCluster=1000)
                KSClusterDiff1000.clusterKS()
    
            # ES_matrix,  ES_dist, ES_Count_matrix = docs_distribution(baseSpace=ES, tE=tE, columns=columns)
            # ES_size = list(range(ES_matrix.shape[0]))
            
            ## We set to 0 the distance here for each recipe -- difference needs to estimate distance between all points. 
            #This serves as optim to not calculate for each varaitions but only once since it is the same distance for all KB
            # print('train variations for recipe {} done'.format(recette), ' | train variation size : ', len(train_recettes))
            
            neighborhood_distance  = 0.
            new_ratio_vec = []
            new_bin_vec = []
            uniq_ratio_vec = []
            uniq_bin_vec = []
            diff_ratio_vec = []
            diff_bin_vec = []
            neighborhood_distance_vec = []
            # mean100_vec = []
    
            for i in (range(len(tE))):
                # print("new KS dist (with 1 toEval)")
                select_variation = KS_size + [len(KS_size)+i]
                NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
                
                # print("new ES pmi (with 1 toEval)")
                # # instance_ES_updated_pmi = copy.deepcopy(incremental_pmi_ES_only) # reprise du pmi du ES seulement
                # new_pmi_instance = OptimizedIncrementalPMI(window_size = w_size)
                # update_text = [word for text in tqdm(tE[i]) for word in text.split()] 
                # new_pmi_instance.update(update_text)
                # new_pmi = new_pmi_instance.compute_pmi()
    
    
                # print("compute scores")
                results = compute_scores(KS_matrix, KS_dist, NewKS_dist, variation_dist,# ES_pmi, dict_ES_pmi, new_pmi,
                                            neighbor_dist=neighborhood_distance, useClusters=True, KSCluster=KSClusterDiff1000)
    
                new_ratio = results[0]
                new_bin = results[1]
    
                uniq_ratio = results[2]
                uniq_bin = results[3]
    
                diff_ratio = results[4]
                diff_bin = results[5]
                neighborhood_distance = results[6] #update neighborhood distance so it isn't reset
                # mean100 = results[7]
    
                # surpNew_ratio = results[7]
                # surpNew_bin = results[8]
    
                # surpDiv_ratio = results[9]
                # surpDiv_bin = results[10]
    
                new_ratio_vec.append(new_ratio)
                new_bin_vec.append(new_bin)
    
                uniq_ratio_vec.append(uniq_ratio)
                uniq_bin_vec.append(uniq_bin)
    
                diff_ratio_vec.append(diff_ratio)
                diff_bin_vec.append(diff_bin)
                neighborhood_distance_vec.append(neighborhood_distance)
                # mean100_vec.append(mean100)
    
                if i % 1000 == 0:
                    current_time = time.time()  # Get the current time
                    elapsed_time = current_time - start_time  # Calculate elapsed time
                    if i % 1000 == 0:
                        print(f"{i} on {len(tE)}, time since last print: {elapsed_time:.2f} seconds")
                    start_time = current_time  # Reset the start time for the next interval
                        # Reset the start time to measure time per iteration
                    start_time = time.time()
    
    
            df = pd.DataFrame({
                "application_number": application_number,
                "label": label,
    
                "new_ratio": new_ratio_vec,
                "new_bin": new_bin_vec,
    
                "uniq_ratio": uniq_ratio_vec,
                "uniq_bin": uniq_bin_vec,
    
                "diff_ratio": diff_ratio_vec,
                "diff_bin": diff_bin_vec, 
                "neighboroud_distance": neighborhood_distance_vec,
                # "mean100": mean100_vec
    
                # "surpNew_ratio": surpNew_ratio,
                # "surpNew_bin": surpNew_bin,
    
                # "surpDiv_ratio":surpDiv_ratio,
                # "surpDiv_bin": surpDiv_bin
            })
            df.to_csv(pathOutput + f'/{year}_{ipc}_{columns[0]}_noveltyMetrics.csv', index=False)
            

measureNov(path, pathOutput, columns, w_size, useClusters, yearList, ipcList)

In [2]:
path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
# path = "/home/edgarlanoue/data/csvCleaned/"

In [4]:
tE_names = get_file_names(path+"tE")
KS_names = get_file_names(path+"KS")
ES_names = get_file_names(path+"ES")

tE_set = set([extract_year_ipc(string) for string in tE_names])
KS_set = set([extract_year_ipc(string) for string in KS_names])
ES_set = set([extract_year_ipc(string) for string in ES_names])

assert(tE_set==KS_set==ES_set)

yearList = [year for year, ipc in list(tE_set)]
ipcList = [ipc for year, ipc in list(tE_set)]

yearList = ["2016"]
ipcList = ["H01L"]

for year in yearList:
    for ipc in ipcList:
        print(f"{year}")
        print(f"    {ipc}")
        tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
        KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
        ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

2016
    H01L


In [5]:
import importlib
import utils
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI

In [6]:
application_number = tE["application_number"]
label = tE["label"]

In [7]:
tE=combine_columns(tE, ["claims"])
KS=combine_columns(KS, ["claims"])
ES=combine_columns(ES, ["claims"])

In [8]:
import importlib
import utils
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI, pmi
from tqdm import tqdm

In [17]:
len(ES)

284544

In [9]:
# base_texts = ' '.join(ES[:1000]).split()
base_texts = [word for text in tqdm(ES) for word in text.split()]
# ES_pmi = pmi(base_texts, w_size=3)
# ES_pmi

# 38s

100%|██████████| 284544/284544 [00:27<00:00, 10331.46it/s]


Pmi le mien vs défaut

In [10]:
instOptPMI = OptimizedIncrementalPMI(3)
instOptPMI.update(base_texts)
dict_optPMI = instOptPMI.compute_pmi()
# 8min 20

100%|██████████| 11075532/11075532 [00:31<00:00, 354886.16it/s]


In [11]:
len(dict_optPMI)

4014866

PMI, les deux sont pareils à 1 secondes prêts... le mien est utile puisqu'il segmente les taches, rendant un PMI futur beaucoup plus rapide à calculer, cependant, pu besoin puisqu'on compare simplement l'intersection entre l'ES et le nouveau

In [74]:
baseTexts_update = [tE[1]]
update_text = [word for text in tqdm(baseTexts_update) for word in text.split()] #' '.join(baseTexts_update).split()

100%|██████████| 1/1 [00:00<00:00, 382.62it/s]


In [75]:
new_pmi = OptimizedIncrementalPMI(window_size=3)
new_pmi.update(update_text)
newpmi_PMI = new_pmi.compute_pmi()

100%|██████████| 155/155 [00:00<?, ?it/s]


In [306]:
# # base_texts_set = set(base_texts)
# # update_text_set = set(update_text)

# base_bigram_set = set(instOptPMI.bigram_counts.keys())

# update_bigram_set = set(new_pmi.bigram_counts.keys())

# common_bigram_set = update_bigram_set & base_bigram_set
# # common_word_set = update_text_set & base_texts_set


# import math

# # 20aine de secondes pour trouver ce qu'ils ont en commun

In [318]:
# base_bigram_set = set(instOptPMI.bigram_counts.keys())

# update_bigram_set = set(new_pmi.bigram_counts.keys())

In [76]:
base_bigram_set2 = set(dict_optPMI.keys())
# update_bigram_set2 = set(newpmi_PMI.keys())

In [327]:
# common_bigram_set2 = update_bigram_set2 & base_bigram_set2

In [137]:
import surprise
from surprise import Surprise
from utils import pmi_to_dict_adj_dict
instSurp = Surprise(newpmi_PMI)
instSurp2 = Surprise(newpmi_PMI)
importlib.reload(utils)
from utils import pmi_to_dict_adj_dict, dict2mat
importlib.reload(surprise)
from surprise import Surprise
from utils import pmi_to_dict_adj_dict
instSurp = Surprise(newpmi_PMI)
instSurp2 = Surprise(newpmi_PMI)

In [78]:
dict_known = pmi_to_dict_adj_dict(dict_optPMI)

w1:  84621
w2:  84621


In [113]:
from divergences import Jensen_Shannon

##### Version longue

In [123]:
def unique_surp_longue(newpmi_PMI, dict_known):
    dict_new = pmi_to_dict_adj_dict(newpmi_PMI)
    # print("get_common_vectors")
    vecotr_tuples = instSurp.get_common_vectors_adj(dict_known, dict_new, epsilon = 0)

    key_list = vecotr_tuples.keys()
    surprise_dists = []
    # print(key_list)
    # print("JSDiv")
    len_surpDist=0
    #i=0
    for entry in (key_list):
        tuple_known, tuple_new = vecotr_tuples[entry]
        
        #### We want only positive PMI score -- no negative values for not going nan or inf values  

        mask = (np.array(tuple_known) != 0).astype(int)
        tuple_known = np.maximum(0., np.array(tuple_known))
        # Apply the mask to tuple_known
        tuple_new = tuple_new * mask

        tuple_known = np.maximum(0., np.array(tuple_known))
        # if i==4:
        #     print(list(tuple_known))
        #     print(sum(tuple_known))
        #     print("FINITO")
        #     print(tuple_new)
        #     print(sum(tuple_new))
        # i+=1
        mask = (np.array(tuple_new) != 0).astype(int)
        tuple_new = np.maximum(0., np.array(tuple_new))
        # Apply the mask to tuple_known
        tuple_known = tuple_known * mask

        if tuple_new.sum():
            len_surpDist+=1
            if tuple_known.sum():
                surprise_dists.append(Jensen_Shannon().JSDiv(tuple_known, tuple_new))
            else: surprise_dists.append(0)

    surprise_score = sum(surprise_dists) / len_surpDist # 2 etant la différence entre nb intersection, et nb nouveau

    return surprise_score

##### Version courte

In [124]:
def unique_surp_courte(newpmi_PMI, base_bigram_set2):
    update_bigram_set2 = set(newpmi_PMI.keys())
    common_bigram_set2 = update_bigram_set2 & base_bigram_set2

    dict_new2 = pmi_to_dict_adj_dict({key: newpmi_PMI[key] for key in common_bigram_set2})
    dict_known2 = pmi_to_dict_adj_dict({key: dict_optPMI[key] for key in common_bigram_set2})
    vecotr_tuples2 = instSurp2.get_common_vectors_adj(dict_known2, dict_new2, epsilon = 0)

    key_list = vecotr_tuples2.keys()
    surprise_dists = []

    for entry in (key_list):
        tuple_known2, tuple_new2 = vecotr_tuples2[entry]
        
        #### We want only positive PMI score -- no negative values for not going nan or inf values  
        mask = (np.array(tuple_known2) != 0).astype(int)
        tuple_known2 = np.maximum(0., np.array(tuple_known2))
        # Apply the mask to tuple_known
        tuple_new2 = tuple_new2 * mask

        mask = (np.array(tuple_new2) != 0).astype(int)
        tuple_new2 = np.maximum(0., np.array(tuple_new2))
        # Apply the mask to tuple_known
        tuple_known2 = tuple_known2 * mask

        if tuple_known2.sum() and tuple_new2.sum():
            surprise_dists.append(Jensen_Shannon().JSDiv(tuple_known2, tuple_new2))
            # print(Jensen_Shannon().JSDiv(tuple_known2, tuple_new2))
        else: surprise_dists.append(0)
    surprise_score2 = sum(surprise_dists) / len(surprise_dists)

    return surprise_score2

##### Version mat

In [138]:
def unique_surp_mat(newpmi_PMI, base_bigram_set2):
    update_bigram_set2 = set(newpmi_PMI.keys())
    common_bigram_set2 = update_bigram_set2 & base_bigram_set2

    mat_old = dict2mat({key: dict_optPMI[key] for key in common_bigram_set2})
    mat_new = dict2mat({key: newpmi_PMI[key] for key in common_bigram_set2})

    surprise_dists3 = []
    # print(key_list)
    # print("JSDiv")
    # i=0
    for entry in (mat_old.columns):
        tuple_known3, tuple_new3 = mat_old[entry], mat_new[entry]
        
        #### We want only positive PMI score -- no negative values for not going nan or inf values  
        mask = (np.array(tuple_known3) != 0).astype(int)
        tuple_known3 = np.maximum(0., np.array(tuple_known3))
        # Apply the mask to tuple_known
        tuple_new3 = tuple_new3 * mask

        # tuple_known3 = np.maximum(0., np.array(tuple_known3))
        # if i==4:
        #     print(list(tuple_known3))
        #     print(sum(tuple_known3))
        #     print("FINITO")
        #     print(list(tuple_new3))
        #     print(sum(tuple_new3))
        # i+=1
        mask = (np.array(tuple_new3) != 0).astype(int)
        tuple_new3 = np.maximum(0., np.array(tuple_new3))
        # Apply the mask to tuple_known
        tuple_known3 = tuple_known3 * mask

        if tuple_known3.sum() and tuple_new3.sum():
            surprise_dists3.append(Jensen_Shannon().JSDiv(tuple_known3, tuple_new3))
            # print(Jensen_Shannon().JSDiv(tuple_known3, tuple_new3))
        else: surprise_dists3.append(0)

    surprise_score3 = sum(surprise_dists3) / (len(surprise_dists3))
    return surprise_score3


In [145]:
surprise_score = unique_surp_longue(newpmi_PMI, dict_known)
surprise_score

0.15938992446016972

In [143]:
surprise_score2 = unique_surp_courte(newpmi_PMI, base_bigram_set2)
surprise_score2

0.15938992446016975

In [144]:
surprise_score3 = unique_surp_mat(newpmi_PMI, base_bigram_set2)
surprise_score3

0.15938992446016975

In [146]:
import timeit

# Define wrapper functions to pass the arguments
def wrapper_func1():
    unique_surp_longue(newpmi_PMI, dict_known)

def wrapper_func2():
    unique_surp_courte(newpmi_PMI, base_bigram_set2)

def wrapper_func3():
    unique_surp_mat(newpmi_PMI, base_bigram_set2)

# Measure execution time for each function
time_func1 = timeit.timeit(wrapper_func1, number=100)  # Run 10,000 iterations
time_func2 = timeit.timeit(wrapper_func2, number=100)
time_func3 = timeit.timeit(wrapper_func3, number=100)

# Print results
print(f"func1 execution time: {time_func1:.6f} seconds")
print(f"func2 execution time: {time_func2:.6f} seconds")
print(f"func3 execution time: {time_func3:.6f} seconds")

# Determine the fastest function
fastest = min(time_func1, time_func2, time_func3)
if fastest == time_func1:
    print("func1 is the fastest.")
elif fastest == time_func2:
    print("func2 is the fastest.")
else:
    print("func3 is the fastest.")


func1 execution time: 4.101536 seconds
func2 execution time: 0.169670 seconds
func3 execution time: 0.706946 seconds
func2 is the fastest.


In [300]:
surprise_score2

0.09356191140192403

In [301]:
surprise_score

0.09356191140192403

In [224]:
import surprise
from surprise import Surprise
importlib.reload(utils)
importlib.reload(surprise)
from utils import pmi_to_dict_adj_dict

In [84]:
instSurp = Surprise(newpmi_PMI)
from utils import pmi_to_dict_adj_dict

In [82]:
dict_known = pmi_to_dict_adj_dict(dict_optPMI)

In [83]:
len(dict_known)

45962

get_common_vectors


100%|██████████| 59/59 [00:00<00:00, 11834.15it/s]


In [87]:
from divergences import Jensen_Shannon 

In [20]:
key_list = vecotr_tuples.keys()
surprise_dists = []
# print(key_list)
print("JSDiv")
#i=0
for entry in tqdm(key_list):
    tuple_known, tuple_new = vecotr_tuples[entry]
    
    #### We want only positive PMI score -- no negative values for not going nan or inf values  
    tuple_known = np.maximum(0., np.array(tuple_known))
    # if i==4:
    #     print(list(tuple_known))
    #     print(sum(tuple_known))
    #     print("FINITO")
    #     print(tuple_new)
    #     print(sum(tuple_new))
    # i+=1
    tuple_new = np.maximum(0., np.array(tuple_new))

    if tuple_known.sum() and tuple_new.sum():
        surprise_dists.append(Jensen_Shannon().JSDiv(tuple_known, tuple_new))
    else: surprise_dists.append(0)

surprise_score = sum(surprise_dists) / len(surprise_dists)

surprise_score

JSDiv


100%|██████████| 59/59 [00:00<?, ?it/s]


0.5196458649677629

In [89]:
surprise_score

0.5251444058852863